In [1]:
import sys
project_root_path = '../../..'
python_lib_path = project_root_path + '/python/lib'
sys.path.append(python_lib_path)
import commit_stats

In [2]:
config_map = {}
config_map['ocean'] = {}
ocean_cfg = config_map['ocean']
ocean_cfg['stats_dir'] = 'sandbox/data/bob/ocean_repos'
ocean_cfg['stat_file_name'] = 'commit_stat.json'
config_map['marine'] = {}
marine_cfg = config_map['marine']
marine_cfg['stats_dir'] = 'data/github/2022-04-07-marine-search/projects'
marine_cfg['stat_file_name'] = 'commit-stat.log.json'

In [3]:
import json
out_path = project_root_path + '/' + 'python/cfg/ds_pipeline.json'
out_handle = open(out_path, "w");
out_handle.write(json.dumps(config_map, indent=2));
out_handle.flush()
out_handle.close()

In [4]:
import json
in_path = project_root_path + '/' + 'python/cfg/ds_pipeline.json'
config_map = json.loads(open(in_path, 'r').read())
config_map

{'ocean': {'stats_dir': 'sandbox/data/bob/ocean_repos',
  'stat_file_name': 'commit_stat.json'},
 'marine': {'stats_dir': 'data/github/2022-04-07-marine-search/projects',
  'stat_file_name': 'commit-stat.log.json'}}

In [5]:
cfg = config_map['ocean']

path = project_root_path + '/' + cfg['stats_dir']
all_commit_stats = commit_stats.load_commit_stats(cfg['stat_file_name'], path)
all_commit_stats[slice(0,5)]

[{'commit': '1d327bf1008917b2b04e03ae43840fa577d397d6',
  'Author': 'mihaisc <mihai.scarlat@smartcontrol.ro>',
  'Date': 'Date:   Fri May 6 11:54:12 2022 +0300',
  'fileTypes': {'md': {'textLineCount': 2884,
    'binByteCount': 0,
    'inserts': 1469,
    'deletes': 1415,
    'occurrences': 1},
   'json': {'textLineCount': 4,
    'binByteCount': 0,
    'inserts': 2,
    'deletes': 2,
    'occurrences': 2}},
  'files': '3',
  'insertions(+)': '1449',
  'deletions(-)': '1439'},
 {'commit': '0e261b4dec0cdc2a30b89feb489677b45b5707e9',
  'Author': 'Matthias Kretschmann <m@kretschmann.io>',
  'Date': 'Date:   Fri May 6 09:44:52 2022 +0100',
  'fileTypes': {'json': {'textLineCount': 118,
    'binByteCount': 0,
    'inserts': 59,
    'deletes': 59,
    'occurrences': 2}},
  'files': '2',
  'insertions(+)': '9',
  'deletions(-)': '109'},
 {'commit': '7782bf49b37cd88bfa000805cdff2bd6de716d5c',
  'Author': 'Matthias Kretschmann <m@kretschmann.io>',
  'Date': 'Date:   Thu May 5 16:42:21 2022 +0100

In [6]:
ft_dicts = []
ft_dict_dict = {}
for stats in all_commit_stats:
    for file_type in stats['fileTypes']:
        commit_ft_dict = stats['fileTypes'][file_type]
        if file_type not in ft_dict_dict:
            ft_dict = {}
            ft_dicts.append(ft_dict)
            ft_dict_dict[file_type] = ft_dict
            ft_dict['file_type'] = file_type
            ft_dict['text_lines'] = 0
            ft_dict['files'] = 0
            ft_dict['commits'] = 0
        ft_dict = ft_dict_dict[file_type]
        ft_dict['commits'] += 1
        ft_dict['text_lines'] += commit_ft_dict['textLineCount']
        ft_dict['files'] += commit_ft_dict['occurrences']


In [7]:
by_commit_count = sorted(ft_dicts, key=lambda d: d['commits'])
by_commit_count.reverse()
# if your odds of being in a commit are < 1/1000, you don't matter
main_type_stats = [stats for stats in by_commit_count if stats['commits'] > (len(all_commit_stats) / 1000)]
print(len(main_type_stats))
main_type_stats

27


[{'file_type': 'ts', 'text_lines': 2534922, 'files': 64314, 'commits': 23634},
 {'file_type': 'py', 'text_lines': 2550398, 'files': 77847, 'commits': 23499},
 {'file_type': 'json',
  'text_lines': 687641548,
  'files': 218806,
  'commits': 20640},
 {'file_type': 'tsx', 'text_lines': 1459422, 'files': 73260, 'commits': 20448},
 {'file_type': 'md', 'text_lines': 1221011, 'files': 16980, 'commits': 13343},
 {'file_type': 'js', 'text_lines': 1288620, 'files': 15534, 'commits': 8172},
 {'file_type': 'sol', 'text_lines': 1263060, 'files': 26604, 'commits': 7992},
 {'file_type': 'css', 'text_lines': 255546, 'files': 22266, 'commits': 7056},
 {'file_type': 'yml', 'text_lines': 65707, 'files': 6497, 'commits': 5946},
 {'file_type': 'cfg', 'text_lines': 11606, 'files': 4220, 'commits': 3801},
 {'file_type': 'ini', 'text_lines': 28876, 'files': 3320, 'commits': 2953},
 {'file_type': 'noexttext',
  'text_lines': 41652,
  'files': 2385,
  'commits': 2148},
 {'file_type': 'sh', 'text_lines': 13524, 

In [8]:
import pandas as pd

main_types = [stat['file_type'] for stat in main_type_stats]
main_types.index('json')
rows = []
for commit_stats in all_commit_stats:
    row = [0]*len(main_types)
    rows.append(row)
    for file_type in commit_stats['fileTypes']:
        commit_ft_dict = commit_stats['fileTypes'][file_type]
        if file_type in main_types:
            position = main_types.index(file_type)
            row[position] = commit_ft_dict['textLineCount']
type_df = pd.DataFrame(rows, columns=main_types) # index=commit_ids,
type_df

,ts,py,json,tsx,md,js,sol,css,yml,cfg,...,yaml,template,eslintrc,png,html,in,jpg,dockerignore,info,abi
0,0,0,4,0,2884,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,118,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,6,0,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,6,0,14,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93955,0,108,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
93956,0,157,0,0,9,0,0,0,60,0,...,0,0,0,0,0,0,0,0,0,0
93957,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
93958,0,470,0,0,0,0,0,0,25,26,...,0,0,0,0,0,9,0,0,0,0


In [9]:
type_corr = type_df.corr()
type_corr

,ts,py,json,tsx,md,js,sol,css,yml,cfg,...,yaml,template,eslintrc,png,html,in,jpg,dockerignore,info,abi
ts,1.000000,-0.013118,-0.002764,0.092188,-0.006145,0.000200,-0.008542,0.048205,0.006680,-0.011566,...,-0.002520,-0.002908,0.093220,NaN,-0.001932,-0.002564,NaN,-0.002987,-0.002737,-0.002409
py,-0.013118,1.000000,0.003899,-0.023087,0.008776,-0.017892,0.001882,-0.014343,0.021046,0.060217,...,-0.001181,0.004156,-0.004752,NaN,-0.003228,0.024749,NaN,0.024549,-0.004574,0.260427
json,-0.002764,0.003899,1.000000,0.000501,-0.004672,0.022419,0.097025,-0.002013,-0.006294,-0.008505,...,-0.001860,-0.002103,-0.000004,NaN,0.001250,-0.001892,NaN,-0.002204,0.000259,0.000923
tsx,0.092188,-0.023087,0.000501,1.000000,0.004174,0.027947,-0.015034,0.633350,0.030812,-0.020356,...,-0.004436,-0.005117,0.163617,NaN,-0.003400,-0.004512,NaN,-0.005257,-0.004818,-0.004239
md,-0.006145,0.008776,-0.004672,0.004174,1.000000,-0.008489,-0.005349,0.014376,-0.003101,-0.005228,...,0.140020,0.003607,0.009468,NaN,0.007897,0.014161,NaN,0.011423,0.073124,0.008845
js,0.000200,-0.017892,0.022419,0.027947,-0.008489,1.000000,0.099913,0.037892,-0.008695,-0.015776,...,-0.003438,-0.003966,0.009685,NaN,0.054250,-0.003497,NaN,0.000550,0.046138,-0.003285
sol,-0.008542,0.001882,0.097025,-0.015034,-0.005349,0.099913,1.000000,-0.006934,-0.010060,-0.012586,...,-0.002742,-0.003164,-0.003094,NaN,0.006820,-0.002789,NaN,-0.003250,0.008466,0.050649
css,0.048205,-0.014343,-0.002013,0.633350,0.014376,0.037892,-0.006934,1.000000,0.050656,-0.012846,...,-0.002760,-0.003229,0.226931,NaN,0.299879,-0.002847,NaN,-0.003317,0.247424,-0.002675
yml,0.006680,0.021046,-0.006294,0.030812,-0.003101,-0.008695,-0.010060,0.050656,1.000000,0.070534,...,0.009014,0.014031,0.022285,NaN,-0.002275,0.079428,NaN,0.104754,-0.003224,-0.002837
cfg,-0.011566,0.060217,-0.008505,-0.020356,-0.005228,-0.015776,-0.012586,-0.012846,0.070534,1.000000,...,0.034030,0.062991,-0.004190,NaN,-0.002846,0.462516,NaN,0.150204,-0.004033,0.012142


In [10]:
for i in range(len(main_types)):
    for j in range(len(main_types)):
        if i < j:
            val = type_corr.iat[i, j]
            if val > 0.1:
                print(main_types[i] + ", " + main_types[j] + ": " + str(val))

ts, noexttext: 0.11974350360726997
py, abi: 0.26042661109619014
json, sh: 0.12029355167664645
tsx, css: 0.6333504150996819
tsx, noexttext: 0.20769407551601118
tsx, sh: 0.13357903719125075
tsx, example: 0.33490055559065
tsx, svg: 0.24273788517405304
tsx, eslintrc: 0.1636165216704317
md, yaml: 0.1400197305639038
css, noexttext: 0.3048218770561591
css, sh: 0.16813020144321644
css, example: 0.3484745144174882
css, svg: 0.4493832421535617
css, eslintrc: 0.22693128480353336
css, html: 0.29987893358163914
css, info: 0.2474240917739878
yml, sh: 0.14579727089303604
yml, dockerignore: 0.10475360357948882
cfg, ini: 0.1326646801740495
cfg, sh: 0.13989711632760118
cfg, gitignore: 0.11522466272544873
cfg, in: 0.4625159406605571
cfg, dockerignore: 0.1502038149890034
ini, noexttext: 0.14250399037304995
ini, sh: 0.11458100767539832
ini, gitignore: 0.10919620786466959
ini, template: 0.4506815320074332
ini, in: 0.25781655534838555
ini, dockerignore: 0.15389996216386048
noexttext, sh: 0.22130316267124692
